---
# Jupyter notebook Bench_Grid_HiRep starts
---

## Importing the packages and checking if they are on system.
---

In [ ]:
# ----------------------------------------------------------------------------
# [Python-3.12] basic import for system check
# ----------------------------------------------------------------------------
import os
import sys
import time
import requests
import multiprocessing
import multiprocess
from collections import defaultdict
from functools import cache
from pathlib import Path
from glob import glob
# ----------------------------------------------------------------------------
# [Data-Plotting]
# ----------------------------------------------------------------------------
try:
    import matplotlib
    MATPLOTLIB_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package matplotlib is not installed on your system, verify or install\n")
    MATPLOTLIB_AVAILABLE = False
try:
    import tqdm
    TQDM_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package tqdm is not installed on your system, verify or install\n")
    TQDM_AVAILABLE = False
try:
    import pandas
    PANDAS_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package pandas is not installed on your system, verify or install\n")
    PANDAS_AVAILABLE = False
try:
    import seaborn
    SEABORN_AVAILABLE = True
except (ImportError, NameError, AttributeError, OSError):
    print(" Python package seaborn is not installed on your system, verify or install\n")
    SEABORN_AVAILABLE = False

## Checking and printing check result
---

In [ ]:
# ----------------------------------------------------------------------------
# [Import-checks]
# ----------------------------------------------------------------------------
print("\n")
print("MATPLOTLIB_AVAILABLE --- installed --->: ", MATPLOTLIB_AVAILABLE)
print("TQDM_AVAILABLE       --- installed --->: ", TQDM_AVAILABLE)
print("PANDAS_AVAILABLE     --- installed --->: ", PANDAS_AVAILABLE)
print("SEABORN_AVAILABLE    --- installed --->: ", SEABORN_AVAILABLE)
print("\n")

## Setting up the main path structure and appending to system path.
---

In [ ]:
# ----------------------------------------------------------------------------
# [Path]
# ----------------------------------------------------------------------------
current_path = str(Path(sys.path[0]) / "..")
if current_path not in sys.path: sys.path.append(current_path)

APP_ROOT          = os.path.join(os.getcwd(), '..','..','..','..')
PROJECTNAME       = ""
POOL_COMPONENTDIR = ""
SOFTWARE          = "N/A"
SQL_DIR           = 'SQLFiles_sql'
APP_DATA_PATH     = "N/A"

DATAPROCINTERCOM  = ""
TBLECNTS_DIR      = ""
SQL_FULLPATH_DIR  = os.path.join(os.getcwd())
# ----------------------------------------------------------------------------
# [SystemPath-Appens]
# ----------------------------------------------------------------------------
sys.path.append(APP_ROOT)
sys.path.append(APP_DATA_PATH)
sys.path.append(DATA_PATH)
sys.path.append(os.path.join(APP_ROOT, '.'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes'))
sys.path.append(os.path.join(APP_ROOT, '.','src','PythonCodes','utils'))

print("Current Path  --->: ", current_path)
print("APP_ROOT      --->: ", APP_ROOT)
print("DATA_PATH     --->: ", DATA_PATH)
print("APP_DATA_PATH --->: ", APP_DATA_PATH)

## Importing the application objects.
---

In [ ]:
# ----------------------------------------------------------------------------
# [Application]
# ----------------------------------------------------------------------------
#Application imports
import src.PythonCodes.DataManage_common
import src.PythonCodes.utils.messageHandler
import src.PythonCodes.utils.Command_line
import src.PythonCodes.DataManage_header

## Instantiating the main objects
---

In [ ]:
version = src.PythonCodes.DataManage_common.DataManage_version()
c = src.PythonCodes.DataManage_common.DataManage_common()
rc = c.get_RC_SUCCESS()
# Getting the log file
logfile = c.getLogfileName()  #getting the name of the global log file
m = src.PythonCodes.utils.messageHandler.messageHandler(logfile = logfile)
# printing the header of Application
src.PythonCodes.DataManage_header.print_Bench_Grid_header(common=c, messageHandler=m)

## Setting the path structure into the main objects
---

In [ ]:
c.setApp_root(APP_ROOT)
c.setData_path(DATA_PATH)
c.setProjectName(PROJECTNAME)
c.setPool_componentdir(POOL_COMPONENTDIR)
c.setSoftware(SOFTWARE)
c.setDataProcInterCom(DATAPROCINTERCOM)
c.setJSon_TableCounts_Dir(TBLECNTS_DIR)
c.setSql_dir(SQL_DIR)
c.setSql_fullPath_dir(SQL_FULLPATH_DIR)

## Starting the program
---

In [ ]:
# --------------------------------------------------------------------------
# [Main-code]
# --------------------------------------------------------------------------
m.printMesgStr("This is the main program      :", c.getCyan(), "Bench_Grid_HiRep.py")

## Main functions
---

### Reinitialising_Paths_and_object_content

In [ ]:
def Reinitialising_Paths_and_object_content(c, m, data_path, b_action, sim_sz):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    # ----------------------------------------------------------------------
    c.setData_path(data_path)
    c.setTarget_File("target.txt")

    target_file_default = c.getTarget_File()
    m.printMesgStr("Default target file           :", c.getMagenta(), target_file_default)

    msg_analysis = c.getTarget_File().split(".txt")[0] + c.undr_scr + \
                    b_action      + c.undr_scr                      + \
                    sim_sz        + c.undr_scr                      + \
                    "batch_files" + c.txt_ext

    c.setTarget_File(str(msg_analysis))
    m.printMesgStr("Target file for analysis      :", c.getMagenta(), c.getTarget_File())

    c.setTargetdir( os.path.join(c.getData_path(), c.getTarget_File()))
    m.printMesgStr("Full Path target file         :", c.getCyan(), c.getTargetdir())

    if Path(c.getTargetdir()).is_file():
        m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), "Exists")
    # ----------------------------------------------------------------------
    return rc
# [end-function]
# --------------------------------------------------------------------------

### getTarget_file_lst

In [ ]:
def getTarget_file_lst(c, m, target_file):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_lst = []
    target_file_dir = []
    try:
        with open(target_file) as file:
            cnt = 0
            for line in file:
                target_file_lst.append(os.path.basename(line).strip())
                target_file_dir.append(os.path.dirname(os.path.realpath(line)).strip())
                cnt += 1
            # [end-For-Loop]
            m.printMesgAddStr("Number of files in target_file : ", c.getYellow(), str(cnt))
        # [end-with]
    except IOError:
        m.printMesgAddStr(" Filename          : ",
                                   c.getCyan(), target_file)
        m.printMesgAddStr("                   : ",
                                   c.getRed(), "cannot be found check"
                                                    " if file exist")
        #exit(c.get_RC_FAIL())
    # [end-try-catch]
    return rc, target_file_lst, target_file_dir
# [end-function]
# --------------------------------------------------------------------------

### getTaget_file_cluster_lst

In [ ]:
def getTarget_file_cluster_lst(c, m, target_file_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    target_file_cluster_lst = []
    for i in range(len(target_file_lst[:])):
        msg = (os.path.join(c.getData_path(), batch_action, simulation_size, str(target_file_lst[i].split(".sh")[0]), target_file_lst[i])).strip()
        if Path(msg).is_file():
            m.printMesgAddStr("[Check]: target file       --->: ", c.getGreen(), c.getMagenta()+ msg + c.getGreen() + " ---> Exists")
            # Now get the output file to analise and put it into a list
            cluster_out_file = msg.split(".sh")[0]+".out"
            # Extracting cluster files that has been benched
            if Path(cluster_out_file).is_file():
                m.printMesgAddStr("[Check]: Cluster file      --->: ", c.getGreen(), c.getYellow()+ cluster_out_file + c.getGreen() + " ---> Exists")
                target_file_cluster_lst.append(cluster_out_file)
            # [end-if]
        # [end-if]
    # [end-for-loop]
    return rc, target_file_cluster_lst
# [end-function]
# --------------------------------------------------------------------------

### readCluster_file_out

In [ ]:
def readCluster_file_out(c, m, target_file_cluster_lst):
    __func__= sys._getframe().f_code.co_name
    rc = c.get_RC_SUCCESS()
    m.printMesgStr("Getting target file list      :", c.getGreen(), __func__)
    end_key = "###############################################"
    bench_dict = {}
    start_key_rep_lst = [
        'Performing benchmark for SU(2), adjoint',
        'Performing benchmark for SU(2), fundamental',
        'Performing benchmark for SU(3), fundamental',
        'Performing benchmark for Sp(4), fundamental'
    ]
    # Starting the parsing of files over the start_key_lst
    # TODO: loop over the representation key
    ikey = 0
    for i in range(len(target_file_cluster_lst[:])):
        #with open(target_file_cluster_lst[i]) as cluster_file:
        cluster_file = open(target_file_cluster_lst[i])
        lines = cluster_file.readlines()
        database_file_len = len(lines)
        print("the number of lines in file "+ str(target_file_cluster_lst[i]) + " is : "+str(database_file_len))
        start_key_su2_adjoint = start_key_rep_lst[ikey]
        appending = False
        cnt = 0
        insert_msg = ""
        for j in range(database_file_len):
            if start_key_su2_adjoint in lines[j].split('\n')[0]:
                appending = True
            if end_key in lines[j].split('\n')[0]:
                appending = False
            elif appending:
                print(lines[j].split('\n')[0])

            # TODO: continue from here

            # [end-if]
        # [end-for-loop [j]]
    # [end-for-loop [i]]
    return rc
# [end-function]
# --------------------------------------------------------------------------

## Lumi
### [BKeeper_run_gpu : small]
---

In [ ]:
machine_name = "Lumi"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [ ]:
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_lumi, target_file_dir_lumi = getTarget_file_lst(c, m, c.getTargetdir())


In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_lumi = getTarget_file_cluster_lst(c, m, target_file_lst_lumi[:])

In [ ]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out         : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                     : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_lumi[:] --->: ", c.getYellow(), target_file_cluster_lst_lumi[:])

In [ ]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc = readCluster_file_out(c, m, target_file_cluster_lst_lumi[:])

## Vega
### [BKeeper_run_gpu : small]
---

In [ ]:
machine_name = "Vega"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [ ]:
# Reinitialising the paths and object content.
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_vega, target_file_dir_vega = getTarget_file_lst(c, m, c.getTargetdir())

In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_vega = getTarget_file_cluster_lst(c, m, target_file_lst_vega[:])

In [ ]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out            : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                        : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_vega[:]    --->: ", c.getYellow(), target_file_cluster_lst_vega[:])

In [ ]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc = readCluster_file_out(c, m, target_file_cluster_lst_vega[:])

## Leonardo
### [BKeeper_run_gpu : small]
---

In [ ]:
machine_name = "Leonardo"
DATA_PATH         = os.path.join('E:','LatticeRuns','Clusters',machine_name,'LatticeRuns')

In [ ]:
# Reinitialising the paths and object content.
batch_action = "BKeeper_run_gpu"
simulation_size="small"
rc = Reinitialising_Paths_and_object_content(c, m, DATA_PATH, batch_action, simulation_size)

In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_lst_leonardo, target_file_dir_leonardo = getTarget_file_lst(c, m, c.getTargetdir())

In [ ]:
# --------------------------------------------------------------------------
# Getting content in the target file
rc, target_file_cluster_lst_leonardo = getTarget_file_cluster_lst(c, m, target_file_lst_leonardo[:])

In [ ]:
# --------------------------------------------------------------------------
# [Data-Extraction]
# --------------------------------------------------------------------------
m.printMesgStr(   "Data extraction cluster out             : ", c.getGreen(), machine_name)
m.printMesgAddStr("Simulation size                         : ", c.getRed(), simulation_size)
m.printMesgAddStr("target_file_cluster_lst_leonardo[:] --->: ", c.getYellow(), target_file_cluster_lst_leonardo[:])

In [ ]:
# --------------------------------------------------------------------------
# Reading in the inout file
rc = readCluster_file_out(c, m, target_file_cluster_lst_leonardo[:])